In [2]:
from __future__ import division, print_function
from keras import backend as K
from keras.applications import inception_v3
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, merge
from keras.layers.core import Activation, Dense, Dropout, Lambda
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from random import shuffle
from scipy.misc import imresize
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import os
%matplotlib inline


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [3]:
## GPU

os.environ["CUDA_VISIBLE_DEVICE"] = "0" # setting wanna use GPU No.
os.system('echo $CUDA_VISIBLE_DEVICE')

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4 # setting GPU in-use memory
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
set_session(tf.Session(config=config))

In [4]:
dfcols = ['img_name']
df = pd.DataFrame(columns=dfcols)

DATA_DIR = ""
IMAGE_DIR = os.path.join(DATA_DIR, "new_new_test_folder")

HOLIDAY_FILE_IDS = os.path.join(DATA_DIR, "holiday-ids.txt")
HOLIDAY_VECS_FILE = os.path.join(DATA_DIR, "holiday-vecs.npy")
BATCH_SIZE = 32 # 128

In [5]:
def get_random_image(img_groups, group_names, gid):
    gname = group_names[gid]
    photos = img_groups[gname]
    pid = np.random.choice(np.arange(len(photos)), size=1)[0]
    pname = photos[pid]
    #if gname+pname =='':
        #print(gid)
        #print(gname)
        #print('empty')
    return gname + pname + ".jpg"

def pair_generator(triples, image_cache, datagens, batch_size=32):
    while True:
        # shuffle once per batch
        indices = np.random.permutation(np.arange(len(triples)))
        num_batches = len(triples) // batch_size
        for bid in range(num_batches):
            batch_indices = indices[bid * batch_size : (bid + 1) * batch_size]
            batch = [triples[i] for i in batch_indices]
            X1 = np.zeros((batch_size, 299, 299, 3))
            X2 = np.zeros((batch_size, 299, 299, 3))
            Y = np.zeros((batch_size, 2))
            l_file , r_file, label_y = [], [], []
            for i, (image_filename_l, image_filename_r, label) in enumerate(batch):
                if datagens is None or len(datagens) == 0:
                    X1[i] = image_cache[image_filename_l]
                    X2[i] = image_cache[image_filename_r]
                else:
                    X1[i] = datagens[0].random_transform(image_cache[image_filename_l])
                    X2[i] = datagens[1].random_transform(image_cache[image_filename_r])
                Y[i] = [1, 0] if label == 0 else [0, 1]
                single_y = [1, 0] if int(label) == 0 else [0, 1]
                l_file.append(image_filename_l)
                r_file.append(image_filename_r)
                label_y.append(single_y)
            yield [X1, X2], label_y, [l_file,r_file]

In [6]:
file_name = []

for img_file in os.listdir(IMAGE_DIR):
    if img_file.find(".jpg") != -1:
        df = df.append(pd.Series([img_file], index=dfcols),
                            ignore_index=True) 
df = df.sort_values("img_name", ascending=True)
df = df.reset_index(drop=True)
df.to_csv('all.csv', encoding='utf-8', index=False)

In [5]:
df.head()

,img_name
0,010001.jpg
1,010002.jpg
2,010003.jpg
3,010010.jpg
4,010011.jpg


In [6]:
print(len(df))

489


In [7]:
group1 = ['0001', '0002', '0003']
# group2 = ['0006', '0007', '0008', '0009']
mlabel = ''
all_triples = []
triples = []
for index_1, row_1 in df.iterrows():
    for index_2, row_2 in df.iterrows():
        if index_2 > index_1:
            if row_1['img_name'][2:6] in group1:
                mg_l = '1'
            else:
                mg_l = '2'
            if row_2['img_name'][2:6] in group1:
                mg_r = '1'
            else:
                mg_r = '2'
            mlabel = '0'
            triples = []
            if (row_1['img_name'][0:2] == row_2['img_name'][0:2] and mg_l == '2' and mg_r == '2') :   
                mlabel = '1'
                triples = [row_1['img_name'],row_2['img_name'],mlabel]
#             if (row_1['img_name'][0:2] != row_2['img_name'][0:2] and mg_l == '2' and mg_r == '2') :   
#                 mlabel = '0'
#                 triples = [row_1['img_name'],row_2['img_name'],mlabel]
            if (row_1['img_name'][0:2] == row_2['img_name'][0:2] and mg_l != mg_r ) :   
                mlabel = '0'
                triples = [row_1['img_name'],row_2['img_name'],mlabel]
            if len(triples) != 0 :    
                all_triples.append(triples)

In [8]:
print(len(all_triples))

3178


In [9]:
# df_pair = pd.DataFrame({'col':all_triples})
df_pair = pd.DataFrame(all_triples)

In [10]:
import random

neg = []
neg = df_pair[df_pair[2]=='0'].values.tolist() # 1425
random.shuffle(neg)
print(len(neg))

1008


In [11]:
df_pair_pos = pd.DataFrame
df_pair_pos = df_pair[df_pair[2]=='1'].sample(n=len(neg))
df_pair_pos.head()

,0,1,2
2507,340023.jpg,340026.jpg,1
1922,260012.jpg,260016.jpg,1
2810,420011.jpg,420026.jpg,1
3021,440011.jpg,440027.jpg,1
915,110013.jpg,110021.jpg,1


In [12]:
pos = []
pos = df_pair_pos.values.tolist() # 3044
pos

[['340023.jpg', '340026.jpg', '1'],
 ['260012.jpg', '260016.jpg', '1'],
 ['420011.jpg', '420026.jpg', '1'],
 ['440011.jpg', '440027.jpg', '1'],
 ['110013.jpg', '110021.jpg', '1'],
 ['330023.jpg', '330026.jpg', '1'],
 ['220015.jpg', '220017.jpg', '1'],
 ['180017.jpg', '180023.jpg', '1'],
 ['340015.jpg', '340023.jpg', '1'],
 ['330015.jpg', '330018.jpg', '1'],
 ['220015.jpg', '220016.jpg', '1'],
 ['090014.jpg', '090019.jpg', '1'],
 ['050021.jpg', '050024.jpg', '1'],
 ['130014.jpg', '130016.jpg', '1'],
 ['330015.jpg', '330020.jpg', '1'],
 ['080012.jpg', '080013.jpg', '1'],
 ['080019.jpg', '080020.jpg', '1'],
 ['020013.jpg', '020015.jpg', '1'],
 ['040011.jpg', '040025.jpg', '1'],
 ['340014.jpg', '340018.jpg', '1'],
 ['020011.jpg', '020013.jpg', '1'],
 ['090011.jpg', '090023.jpg', '1'],
 ['040016.jpg', '040018.jpg', '1'],
 ['410014.jpg', '410015.jpg', '1'],
 ['090018.jpg', '090021.jpg', '1'],
 ['410018.jpg', '410020.jpg', '1'],
 ['330018.jpg', '330027.jpg', '1'],
 ['330012.jpg', '330027.jpg'

In [13]:
df_pair.to_csv('all_pair.csv', encoding='utf-8', index=False)

In [8]:
def load_image_cache(image_cache, image_filename):
    image = plt.imread(os.path.join(IMAGE_DIR, image_filename))
    image = imresize(image, (299, 299))
    image = image.astype("float32")
    image = inception_v3.preprocess_input(image)
    image_cache[image_filename] = image
    
image_cache = {}
num_pairs = len(all_triples)
for i, (image_filename_l, image_filename_r, _) in enumerate(all_triples):
    if i % 1000 == 0:
        print("images from {:d}/{:d} pairs loaded to cache".format(i, num_pairs))
    if image_filename_l not in image_cache:
        load_image_cache(image_cache, image_filename_l)
    if image_filename_r not in image_cache:
        load_image_cache(image_cache, image_filename_r)
print("images from {:d}/{:d} pairs loaded to cache, COMPLETE".format(i, num_pairs))

images from 0/3178 pairs loaded to cache


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


images from 1000/3178 pairs loaded to cache
images from 2000/3178 pairs loaded to cache
images from 3000/3178 pairs loaded to cache
images from 3177/3178 pairs loaded to cache, COMPLETE


In [11]:
BEST_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-b-final_100_args6_Nadam.h5")
# FINAL_MODEL_FILE = os.path.join(DATA_DIR, "models", "A_A_inceptionv3-l1-ft-best_100.h5")

print("==== Evaluation Results: best model on test set ====")
best_model = load_model(BEST_MODEL_FILE)
# best_model = load_model(FINAL_MODEL_FILE)


==== Evaluation Results: best model on test set ====


OSError: Unable to open file (unable to open file: name = 'models/A_A_inceptionv3-l1-b-final_100_args6_Nadam.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
i = 0

ytest, ytest_ , y_prob, l_file, r_file= [], [], [], [], []
test_pair_gen = pair_generator(all_triples, image_cache, None, BATCH_SIZE)
num_test_steps = len(all_triples) // BATCH_SIZE
print('total batch :', num_test_steps)
curr_test_steps = 0
for [X1test, X2test], Ytest, [file_l,file_r] in test_pair_gen:
    if curr_test_steps > num_test_steps:
        break
    Ytest_ = best_model.predict([X1test, X2test])
    ytest.extend(np.argmax(Ytest, axis=1).tolist())
    ytest_.extend(np.argmax(Ytest_, axis=1).tolist())
    y_prob.extend(Ytest_)
    l_file.extend(file_l)
    r_file.extend(file_r)

    curr_test_steps += 1
    i = i +1
    print(i)
acc = accuracy_score(ytest, ytest_)
cm = confusion_matrix(ytest, ytest_)
    
print("Accuracy Score: {:.3f}".format(acc))
print("Confusion Matrix")
print(cm)

total batch : 99
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Accuracy Score: 0.844
Confusion Matrix
[[ 529  486]
 [  14 2171]]


### A_A_inceptionv3-l1-f-final_100_args6.h5
@ new_new_test_folder

```
Accuracy Score: 0.843
Confusion Matrix
[[ 516  500]
 [   4 2180]]
 ```

### A_A_inceptionv3-l1-f-best_100_args6.h5
@ new_test_folder

```
Accuracy Score: 0.856
Confusion Matrix
[[ 939  489]
 [ 157 2895]]
```

@ new_new_test_folder

```
Accuracy Score: 0.881
Confusion Matrix
[[ 694  321]
 [  60 2125]]
```

```
Accuracy Score: 0.843
Confusion Matrix
[[1009  428]
 [ 276 2767]]
 ```

In [18]:
file_1 = DataFrame(l_file)
file_1.columns = ['l_filename']
file_r = DataFrame(r_file)
file_r.columns = ['r_filename']
mlabel = DataFrame(ytest)
mlabel.columns = ['Label']
pre_label = DataFrame(ytest_)
pre_label.columns = ['Pred_Label']
pro_label = DataFrame(y_prob)
pro_label.columns = ['net_pro','pos_pro']


bbb = pd.concat([file_1,file_r,mlabel,pre_label,pro_label] , axis = 1)

In [19]:
bbb.to_csv('result881.csv', encoding='utf-8', index=False)